In [1]:
import numpy as np
import pandas as pd
import cupy as cp

from nariflow import Variable
from nariflow import optimizer
from nariflow import GradientTape
#from nariflow import calc_gradient
from nariflow import layer
from nariflow.models import Model
from nariflow import functions as f
from nariflow.core import elementary_function as ef
from nariflow.core import shape_function as sf
from nariflow.utils import unit_test
import numpy as np
import pandas as pd

import tensorflow as tf

import time

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject


In [3]:
test_module = unit_test.UnitTest()

In [4]:
test_module.start_testing()

high_order_test_0  : ok
high_order_test_1  : ok
matrix_order_test_0  : ok
matrix_order_test_1  : ok
matrix_order_test_2  : ok
matmul  : ok
reduce_sum  : ok
div  : ok
sum  : ok
mul  : ok
gradient_start_middle  : ok
gradient_stop_test  : ok
loss_value : [0.13590743296422317, 6.346745396681533e-05, 6.346745396681533e-05, 6.346745396681533e-05, 6.346745396681533e-05, 6.346745396681533e-05]
model is under local minima, Attempt to retry..
loss_value : [0.134348352688869, 6.293122714453656e-05, 4.849686354375728e-06, 3.5332563999898783e-06, 3.0236138204508404e-06, 3.914724236947594e-06]
model is under local minima, Attempt to retry..
loss_value : [0.1307470589287228, 5.617119216644496e-06, 4.2700448076255035e-06, 2.93011787081548e-06, 2.035408299672274e-06, 1.4882193457970355e-06]
model training test : ok


## CORE_TAPE

In [61]:
class Variable():
    def __init__(self, data):
        self.data = data
        self.generation = 0
        self.grad = None

    def set_generation(self, generation):
        self.generation = generation + 1

    def resetgrad(self):
        self.grad = None

    def shape(self):
        return self.data.shape

    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

In [62]:

class Function():
    def __call__(self, *inputs):
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x

        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        inputs = [as_variable(as_array(x)) for x in inputs]
        x_list = [i.data for i in inputs]
        y_list = self.forward(*x_list)
        if not isinstance(y_list, tuple):
            y_list = (y_list,)

        output_list = [Variable(as_array(y)) for y in y_list]
        generation = np.max([i.generation for i in inputs])
        for output in output_list:
            output.set_generation(generation)
        self.generation = generation

        if 'GRADIENT_NUM' in globals():
            GRADIENT_NUM = globals()['GRADIENT_NUM']
            self.making_gradient_tape(output_list, inputs)

        if len(output_list) > 1:
            return output_list
        else:
            return output_list[0]

    def making_gradient_tape(self, output, inputs):
        for i in output:
            GRADIENT_NUM = globals()['GRADIENT_NUM']
            for j in range(GRADIENT_NUM + 1):
                globals()[f'GRADIENT_TAPE_{j}'][i] = (self, inputs, self.generation)

    def forward(self, x_list):
        raise NotImplementedError()

    def backward(self, gy_list):
        raise NotImplementedError()

In [63]:

class GradientTape():

    def __init__(self):
        if 'GRADIENT_NUM' not in globals():
            globals()['GRADIENT_NUM'] = 0
        else :
            globals()['GRADIENT_NUM'] += 1
        GRADIENT_NUM = globals()['GRADIENT_NUM']
        globals()[f'GRADIENT_TAPE_{GRADIENT_NUM}'] = dict()
        self.gradient_tape = globals()[f'GRADIENT_TAPE_{GRADIENT_NUM}']
        self.gradient_num = globals()['GRADIENT_NUM']

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        if f'GRADIENT_TAPE_{self.gradient_num}' in globals():
            del globals()[f'GRADIENT_TAPE_{self.gradient_num}']
        if 'GRADIENT_NUM' in globals():
            del globals()['GRADIENT_NUM']
        return

    def CalcGradient(self, target = None, tapes = None, resetgrad = False):
        if tapes is None:
            tapes = self.gradient_tape
        tapes = dict(sorted(tapes.items(), key = lambda x : x[1][2]))

        if target is not None:
            target_ind = [i for i, j in enumerate([i == target for i in list(tapes)]) if j][0]
            tapes_dict = dict()
            [tapes_dict.update(i) for i in [{i[0]: i[1]} for i in tapes.items()][0:target_ind + 1]]
            tapes = dict(reversed(tapes_dict.items()))
        else :
            tapes = dict(reversed(tapes.items()))

        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x

        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)

        for tape in tapes.items():
            outputs = tape[0]
            generation = tape[1][2]
            inputs = tape[1][1]
            func = tape[1][0]

            if isinstance(outputs, Variable):
                outputs = [outputs]

            for j in outputs:
                if j.grad is None:
                    j.grad = Variable(np.ones_like(j.data))

            gy_list = [output.grad for output in outputs]
            func.input_list = inputs
            gx_list = func.backward(*gy_list)
            if not isinstance(gx_list, tuple):
                gx_list = (gx_list,)

            for x, gx in zip(inputs, gx_list):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
            if resetgrad:
                self.resetgrads()

    def resetgrads(self):
        for i in list(self.gradient_tape.keys()):
            i.grad = None

        for i in list(self.gradient_tape.values()):
            for j in i[1]:
                j.grad = None

    def jacobian(self, target = None, tapes = None, var = None, var_return = 'Variable'):
        if tapes is None:
            tapes = self.gradient_tape.copy()
        tapes = dict(sorted(tapes.items(), key=lambda x: x[1][2]))

        if target is not None:
            target_ind = [i for i, j in enumerate([i == target for i in list(tapes)]) if j][0]
            tapes_dict = dict()
            [tapes_dict.update(i) for i in [{i[0]: i[1]} for i in tapes.items()][0:target_ind + 1]]
            tapes = dict(reversed(tapes_dict.items()))
        else :
            tapes = dict(reversed(tapes.items()))
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            if isinstance(x, type(np.array([]))):
                if len(x.shape) == 0:
                    return np.array([[x]])
            return x

        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        if len(list(tapes.keys())[0].data.shape) >= 2:
            i_max = list(tapes.keys())[0].data.shape[0]
            j_max = list(tapes.keys())[0].data.shape[1]
        else :
            i_max = 1
            j_max = 1

        jacobian_dict = dict()
        for jacobian_iter_i in range(i_max):
            jacobian_dict_j = dict()
            for jacobian_iter_j in range(j_max):
                temp_dict = dict()
                for tape in tapes.items():
                    outputs = tape[0]
                    generation = tape[1][2]
                    inputs = tape[1][1]
                    func = tape[1][0]


                    if isinstance(outputs, Variable):
                        outputs = [outputs]

                    for j in outputs:
                        if j.grad is None:
                            j.data = as_array(j.data)
                            grad_matrix = np.zeros_like(j.data)
                            grad_matrix[jacobian_iter_i][jacobian_iter_j] = 1
                            j.grad = Variable(grad_matrix)

                    gy_list = [output.grad for output in outputs]
                    func.input_list = inputs
                    gx_list = func.backward(*gy_list)
                    if not isinstance(gx_list, tuple):
                        gx_list = (gx_list,)
                    for x, gx in zip(inputs, gx_list):
                        if x.grad is None:
                            x.grad = gx
                        else:
                            x.grad = x.grad + gx
                        temp_dict[x] = x.grad.data
                self.resetgrads()
                jacobian_dict_j[jacobian_iter_j] = temp_dict
            jacobian_dict[jacobian_iter_i] = jacobian_dict_j
        if var is None:
            return jacobian_dict

        selected_jacobian = list()
        for i in jacobian_dict:
            for j in jacobian_dict[i]:
                selected_jacobian.append(jacobian_dict[i][j][var])
        if var_return == 'numpy':
            return np.array(selected_jacobian)
        if var_return == 'list':
            return selected_jacobian
        if var_return == 'Variable':
            return [as_variable(x) for x in selected_jacobian]
        else :
            raise Exception('var_return only accpet "numpy", "list" or "Variable"')

# 함수

## 미분 기본 공식

In [64]:
from nariflow.thirdparty.functions import reshape_sum_backward
import numpy as np

# 덧셈
class Add(Function):
    # 정전파 : 두 변수를 더한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 + x_1
        return y

    # 역전파 : 뒷 단계에서 들어온 그레디언트를 양쪽으로 균등하게 흘려보낸다.
    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = shape_f.sumto(gx0, self.x_0_shape)
            gx1 = shape_f.sumto(gx1, self.x_1_shape)
        return gx0, gx1


# 곱셈
class Mul(Function):
    # 정전파 : 두 변수를 곱한다.
    def forward(self, x_0, x_1):
        y = x_0 * x_1
        return y

    # 역전파 : 방향을 스위치해서 뒷 단계의 그레디언트와 입력 변수를 곱해 흘려보낸다.
    def backward(self, gy):
        x_0 = self.input_list[0]
        x_1 = self.input_list[1]
        self.x_0_shape = x_0.data.shape
        self.x_1_shape = x_1.data.shape
        gx0, gx1 = gy, gy
        x0 = x_1 * gx1
        x1 = x_0 * gx0
        if self.x_0_shape != self.x_1_shape:
            x0 = shape_f.sumto(x0, self.x_0_shape)
            x1 = shape_f.sumto(x1, self.x_1_shape)
        return x0, x1

    # 음수 변환


class Neg(Function):
    # 정전파 : 음수로 바꾼다.
    def forward(self, x):
        return -x

    # 역전파 : 음수로 바꿔 흘려보낸다.
    def backward(self, gy):
        return -gy


# 뺄셈
class Sub(Function):
    # 정전파 : 두 변수를 뺀다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 - x_1
        return y

    # 역전파 : 앞 변수는 그레디언트를, 뒤 변수는 그레디언트 음수를 흘려보낸다.
    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = shape_f.sumto(gx0, self.x_0_shape)
            gx1 = shape_f.sumto(gx1, self.x_1_shape)
        return gx0, -gx1


# 나눗셈
class Div(Function):
    # 정전파 : 변수간 나눗셈을 구한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 / x_1
        return y

    # 역전파 : 앞 변수의 경우 1 / a를, 뒤 변수의 경우 (- a / b **2)를 그레디언트와 곱해 흘려보낸다.
    def backward(self, gy):
        x_0, x_1 = self.input_list
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = shape_f.sumto(gx0, self.x_0_shape)
            gx1 = shape_f.sumto(gx1, self.x_1_shape)
        gx_0 = (1 / x_1) * gx0
        gx_1 = (- x_0 / (x_1) ** 2) * gx1
        return gx_0, gx_1

    # 거듭제곱


class Pow(Function):
    # Function 클래스에 거듭제곱 수를 init으로 정의한다.
    def __init__(self, power):
        self.power = power

    # 정전파 : 변수에 거듭제곱을 한다.
    def forward(self, x):
        y = x ** self.power
        return y

    # 역전파 : power * x ^ (power - 1) 에 그레디언트를 곱해 흘려보낸다.
    def backward(self, gy):
        x = self.input_list[0]
        gx = self.power * x ** (self.power - 1) * gy
        return gx


def add(x_0, x_1):
    return Add()(x_0, x_1)


def mul(x_0, x_1):
    return Mul()(x_0, x_1)


def neg(x):
    return Neg()(x)


def sub(x_0, x_1):
    return Sub()(x_0, x_1)


def rsub(x_0, x_1):
    return Sub()(x_1, x_0)


def div(x_0, x_1):
    return Div()(x_0, x_1)


def rdiv(x_0, x_1):
    return Div()(x_1, x_0)


def power(x, power):
    return Pow(power)(x)

class Sum(Function):
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims
        self.x_shape = None

    def forward(self, x):
        self.x_shape = x.shape
        y = np.sum(x, axis=self.axis, keepdims=self.keepdims)
        return y

    def backward(self, gy):
        gy, shape = reshape_sum_backward(gy,
                              x_shape=self.x_shape,
                              axis=self.axis,
                              keepdims=self.keepdims)
        gy = shape_f.reshape(gy, shape)
        gx = shape_f.broadcast_to(gy, self.x_shape)
        return gx

class MatMul(Function):
    def forward(self, x, w):
        y = x.dot(w)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        w = self.input_list[1]
        gw = matmul(shape_f.transpose(x), gy)
        gx = matmul(gy, shape_f.transpose(w))
        return gx, gw

def flowsum(x, axis = None, keepdims = False):
    return Sum(axis, keepdims)(x)

def matmul(x, w):
    return MatMul()(x, w)

class Parameter(Variable):
    pass


# 연산 기본 메소드들을 덮어씌워
# Variable과 연관된 연산은 기호(+, -, *, /)만 사용해도
# 우리가 정의한 연산을 수행하도록 대치한다.
def setup_variable():
    Variable.__add__ = add
    Variable.__radd__ = add
    Variable.__mul__ = mul
    Variable.__rmul__ = mul
    Variable.__neg__ = neg
    Variable.__sub__ = sub
    Variable.__rsub__ = rsub
    Variable.__truediv__ = div
    Variable.__rtruediv__ = rdiv
    Variable.__pow__ = power
    Variable.__getitem__ = shape_f.get_item

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = exp(x) * gy
        return gx

class Log(Function):
    def forward(self, x):
        y = np.log(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = (1 / x) * gy
        return gx

def exp(x):
    return Exp()(x)

def log(x):
    return Log()(x)

class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = cos(x) * gy
        return gx

def sin(x):
    return Sin()(x)

class Cos(Function):
    def forward(self, x):
        y = np.cos(x)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        return -sin(x) * gy

def cos(x):
    return Cos()(x)

class StopGradient(Function):
    def forward(self, x):
        return x

    def backward(self, gy):
        return 0

def stop_gradient(x):
    return StopGradient()(x)

## 텐서 형상 함수

In [65]:

from nariflow.thirdparty.functions import sum_to
import numpy as np


class Reshape(Function):
    def __init__(self, shape):
        # 변환을 원하는 모양을 지정한다.
        self.shape = shape

    def forward(self, x):
        # 원본 모양을 저장해준다.
        self.x_shape = x.shape
        # 모양을 reshpae로 바꾼다.
        y = x.reshape(self.shape)
        return y

    def backward(self, gy):
        # 역전파시에는 원본 모양을 복원한다.
        gx = reshape(gy, self.x_shape)
        return gx


def reshape(x, shape):
    return Reshape(shape)(x)


class Transpose(Function):
    def __init__(self, shape=None):
        self.shape = shape

    def forward(self, x):
        # 원본 텐서의 모양을 저장해둔다.
        # transpose를 실시한다.
        y = np.transpose(x, self.shape)
        return y

    def backward(self, gy):
        if self.shape is None:
            gx = transpose(gy)
        # Variable 변수를 받으므로, np.transpose가 아닌 GoteoFlow의 transpose를 받는다.
        # 저장되어있던 원본 모양을 복원한다.
        else:
            axes_len = len(self.shape)
            inv_axes = tuple(np.argsort([ax % axes_len for ax in self.shape]))
            gx = transpose(gy, shape=inv_axes)
        return gx

def transpose(x, shape=None):
    return Transpose(shape)(x)

class SumTo(Function):
    def __init__(self, shape):
        # 모양을 바꾸면서 덧연산을 실시할 목표 모양을 지정한다.
        self.shape = shape

    def forward(self, x):
        # 원본 모양을 기억한다.
        # sum_to 함수로 모양을 바꾸며 합을 실시한다.
        y = sum_to(x, self.shape)
        return y

    def backward(self, gy):
        self.x_shape = self.input_list[0].data.shape
        # 역전파시엔 sum_to로 인해 바뀌었던 모양을 원본 모양으로 복원한다.
        gx = broadcast_to(gy, self.x_shape)
        return gx


def sumto(x, shape):
    return SumTo(shape)(x)


class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        gx = sumto(x, self.x_shape)
        return gx


def broadcast_to(x, shape):
    return BroadcastTo(shape)(x)

class GetItem(Function):
    def __init__(self, slices):
        self.slices = slices

    # 정전파 : 슬라이싱을 수행한다.
    def forward(self, x):
        y = x[self.slices]
        return y

    # 역전파 : 입력 크기와 슬라이싱 정보를 역함수인 GetItemGrad에 전달한다.
    def backward(self, gy):
        x = self.input_list[0]
        f = GetItemGrad(self.slices, x.data.shape)
        return f(gy)

def get_item(x, slices):
    return GetItem(slices)(x)

#GetItemGrad는 GetItem의 역함수다.
class GetItemGrad(Function):
    def __init__(self, slices, shape):
        self.slices = slices
        self.shape = shape

    #정전파(슬라이싱의 역전파) : 입력 크기만큼의 0행렬을 생성한 후, 슬라이싱 위치에 gy를 채워 반환한다.
    # 슬라이싱에서 잘린 성분은 0 그대로 남는다.
    def forward(self, gy):
        gx = np.zeros(self.shape)
        np.add.at(gx, self.slices, gy)
        return gx

    # 역전파(슬라이싱) : 슬라이싱을 수행한다.
    def backward(self, ggx):
        return get_item(ggx, self.slices)

In [66]:
a = Variable(np.array([3.0,2.0,3.0]))
b = Variable(np.array([3.0,2.0,3.0]))
c = Variable(np.array([1.0,2.0,3.0]))

In [67]:
a = Variable(np.array([[1,3],[4,5], [6,8]]))
b = Variable(np.array([[10,11],[13,14],[15,16]]))

## 테스트 함수

In [3]:
class TestFunction():
    def matyas(self, x, y):
        z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
        return z

    def goldstein(self, x,y):
        z = (1 + (x + y + 1) ** 2 * (19 - 14 * x + 3 * x **2 - 14 * y + 6 * x * y + 3 * y ** 2)) * \
        (30 + (2 * x - 3 * y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
        return z
    
class OrderFunction():
    def high_order_function(self, x, y):
        z = x ** 4 + y ** 3 + x ** 2 + y ** (1/2)
        return z
    def matrix_order_function(self, x):
        y = 2 * x ** 3
        return y
    
class JacobianFunction():
    def matmul(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
        return tape
    
    def reduce_sum(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = ef.flowsum(result_2)
        return tape
    
    def div(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 / result
        return tape
    
    def sum(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 + result
        return tape
    
    def mul(self, x, y, k):
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(y))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 * result
        return tape

class GradientStartFunction():
    def gradient_start_middle(self, x, y, k):
        x = Variable(np.array([[1.,2.],[4.,5.]]))
        v = Variable(np.array([[4.,5.],[6.,7.]]))
        k = Variable(np.array([[1.,3.],[4.,6.]]))
        
        with GradientTape() as tape:
            result = ef.matmul(x,sf.transpose(v))
            result_2 = ef.matmul(result, k)
            result_3 = result_2 / x
        
        tape.CalcGradient(target = result_2)
        return x.grad.data, v.grad.data, k.grad.data
    
    def gradient_stop_test(self, x, y, k):
        x = Variable(np.array([[1.,2.],[4.,5.]]))
        v = Variable(np.array([[4.,5.],[6.,7.]]))
        k = Variable(np.array([[1.,3.],[4.,6.]]))
        
        with GradientTape() as tape_1, GradientTape() as tape_2:
            result = ef.matmul(x,sf.transpose(v))
            result_2 = ef.matmul(ef.stop_gradient(result), k)
            result_2 = result_2 ** 2
            result_3 = result_2 * x
        tape_2.CalcGradient()
        return x.grad.data, k.grad.data

In [4]:
class TestAnswer():
    
    def matyas(self, x = None, y = None):
        if (x is None) | (y is None):
            return (0.040000000000000036, 0.040000000000000036)
        else :
            return

    def goldstein(self, x = None, y = None):
        if (x is None) | (y is None):
            return (-5376.0, 8064.0)
        else :
            return
        
    def high_order_function(self, order, x = None, y = None):        
        if order == 0:
            if (x is None) | (y is None):
                return 36., 12.3535
            else :
                return
            
        if order == 1:
            if (x is None) | (y is None):
                return 50, 11.911
            else :
                return
    
        else :
            return
    
    def matrix_order_function(self, order, x = None, y = None):
        if order == 0:
            if (x is None) | (y is None):
                return np.array([[  6.,  24.], [ 96., 150.]])
            else :
                return
            
        if order == 1:
            if (x is None) | (y is None):
                return np.array([[12., 24.], [48., 60.]])
            else :
                return
    
        if order == 2:
            if (x is None) | (y is None):
                return np.array([[12., 12.], [12., 12.]])
            else :
                return
            
    def matmul(self, x = None, y = None, k = None):
        answer = []
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            for i in ['x','y','k']:
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)

                answer.append(tape.jacobian(result_2, eval(i)).numpy())
            return tuple(answer)
        
    def reduce_sum(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']:        
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = tf.math.reduce_sum(result_2)

                answer.append(tape.jacobian(result_3, eval(i)).numpy())
            
            return tuple(answer)
        
    def div(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']: 
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = result_2 / result

                answer.append(tape.jacobian(result_3, eval(i)).numpy())

            return tuple(answer)
    
    def sum(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']:             
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = result_2 + result

                answer.append(tape.jacobian(result_3, eval(i)).numpy())
            
            return tuple(answer)
        
    def mul(self, x = None, y = None, k = None):
        if x is None:
            x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
            y = tf.Variable(np.array([[4.,5.],[6.,7.]]))
            k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
            answer = []
            for i in ['x','y','k']:             
                with tf.GradientTape() as tape:
                    result = tf.matmul(x,tf.transpose(y))
                    result_2 = tf.matmul(result, k)
                    result_3 = result_2 * result

                answer.append(tape.jacobian(result_3, eval(i)).numpy())
            
            return tuple(answer)
        
    def start_middle(self, x = None, y = None, k = None):
        x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
        v = tf.Variable(np.array([[4.,5.],[6.,7.]]))
        k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
        
        with tf.GradientTape() as tape:
            result = tf.matmul(x,tf.transpose(v))
            result_2 = tf.matmul(result, k)
            result_3 = result_2 / x
            
        answer = tape.gradient(result_2, [x, v])
        
        return answer
    
    def gradient_start_middle(self, x = None, y = None, k = None):
        x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
        v = tf.Variable(np.array([[4.,5.],[6.,7.]]))
        k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
        
        with tf.GradientTape() as tape:
            result = tf.matmul(x,tf.transpose(v))
            result_2 = tf.matmul(result, k)
            result_3 = result_2 / x
            
        answer = tape.gradient(result_2, [x, v, k])
        answer = [i.numpy() for i in answer]
        
        return answer
    
    def gradient_stop_test(self, x = None, y = None, k = None):
        x = tf.Variable(np.array([[1.,2.],[4.,5.]]))
        v = tf.Variable(np.array([[4.,5.],[6.,7.]]))
        k = tf.Variable(np.array([[1.,3.],[4.,6.]]))
        
        with tf.GradientTape() as tape:
            result = tf.matmul(x,tf.transpose(v))
            result_2 = tf.matmul(tf.stop_gradient(result), k)
            result_2 = result_2 ** 2
            result_3 = result_2 * x
            
        answer = tape.gradient(result_3, [x, k])
        answer = [i.numpy() for i in answer]
        
        return answer
        

In [5]:
class Models(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l2 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l3 = layer.Linear(out_size, initializer_func='he_uniform')
        
    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(y)
        y = f.activation.relu(y)
        y = self.l3(y)
        return y
    


In [192]:
class UnitTest():
    def __init__(self):
        self.dataset_path = tf.keras.utils.get_file(
            "auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
        self.jacobian_preset = JacobianFunction()
        self.function_preset = TestFunction()
        self.order_preset = OrderFunction()
        self.start_preset = GradientStartFunction()
        self.answer_preset = TestAnswer()
        
    def data_preprocessing(self):
        def norm(x):
            return (x - train_stats['mean']) / train_stats['std']
        column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                        'Acceleration', 'Model Year', 'Origin']
        raw_dataset = pd.read_csv(self.dataset_path, names=column_names,
                              na_values = "?", comment='\t',
                              sep=" ", skipinitialspace=True)

        dataset = raw_dataset.copy()
        dataset = dataset.dropna()
        origin = dataset.pop('Origin')
        
        dataset['USA'] = (origin == 1)*1.0
        dataset['Europe'] = (origin == 2)*1.0
        dataset['Japan'] = (origin == 3)*1.0
        
        train_dataset = dataset.sample(frac=0.8,random_state=0)
        test_dataset = dataset.drop(train_dataset.index)
        
        train_stats = train_dataset.describe()
        train_stats.pop("MPG")
        train_stats = train_stats.transpose()
        
        normed_train_data = norm(train_dataset)
        normed_test_data = norm(test_dataset)
        train_labels = train_dataset.pop('MPG')
        
        X = Variable(np.array(normed_train_data.drop('MPG',axis = 1)))
        y = (np.array(train_labels) - np.mean(np.array(train_dataset))) / np.std(np.array(train_dataset))
        y = Variable(np.array(y).reshape([-1,1]))
        
        return X,y
    
    def answer_correction(self, function, answer, pred, tor):
        if len(np.array(answer).shape) > 1:
            answer = [i.reshape(j.shape) for i,j in zip(answer, pred)]
        if np.all(np.abs(answer) - np.abs(pred) < tor):
            print(function, ' : ok')
        else :
            print(function, ' : Failed')
            print('answer :', np.abs(answer))
            print('pred :', np.abs(pred))
            print('error :', np.abs(answer) - np.abs(pred))        

    def preset_test(self, tor = 0.01, function = None, x = None, y = None):
        try: 
            if x is None:
                x = Variable(np.array(1.0))
            if y is None:
                y = Variable(np.array(1.0))
            function_list = [i for i in self.function_preset.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = self.function_preset.__getattribute__(function)
                with GradientTape() as tape:
                    z = current_function(x, y)
                tape.CalcGradient()
                pred = (x.grad.data, y.grad.data)
                answer = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answer, pred, tor)
                tape.resetgrads()
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e)
            
    def high_order_test(self, orders = 2):
        try:        
            x = Variable(np.array(2.0))
            y = Variable(np.array(2.0))
            order_function = self.order_preset.high_order_function
            tape_dict = dict()
            with GradientTape() as tape:
                f = order_function(x, y)
            tape_dict[0] = tape
            for order in range(orders):
                with GradientTape() as tape_1:
                    tape_dict[order].CalcGradient()
                tape_dict[order + 1] = tape_1
                pred = (x.grad.data, y.grad.data)
                tape_dict[order].resetgrads()
                answer = self.answer_preset.high_order_function(order)
                self.answer_correction(f'high_order_test_{order}', answer, pred, 0.01)
                tape.resetgrads()
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e)
            
    def matrix_test(self, orders = 3):
        try:
            X = Variable(np.array([[1.,2.],[4.,5.]]))
            matrix_function = self.order_preset.matrix_order_function
            tape_dict = dict()
            with GradientTape() as tape:
                result = matrix_function(X)
            tape_dict[0] = tape
            for order in range(orders):    
                with GradientTape() as tape_1:
                    tape_dict[order].CalcGradient()
                tape_dict[order + 1] = tape_1
                pred = X.grad.data
                tape_dict[order].resetgrads()
                answer = self.answer_preset.matrix_order_function(order)
                self.answer_correction(f'matrix_order_test_{order}',
                                      answer,
                                      pred,
                                      0.01)
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e)
            
    def jacobian_test(self, tor = 0.01, x = None, v = None, k = None, target = None):
        try :
            if x is None:
                x = Variable(np.array([[1.,2.],[4.,5.]]))
            if v is None:
                v = Variable(np.array([[4.,5.],[6.,7.]]))
            if k is None:
                k = Variable(np.array([[1.,3.],[4.,6.]]))
            jacobian_function = self.jacobian_preset
            function_list = [i for i in jacobian_function.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = jacobian_function.__getattribute__(function)
                tape = current_function(x,v,k)
                if target is not None:
                    pred = (tape.jacobian(target = target, var = x, var_return = 'numpy'),
                     tape.jacobian(target = target, var = v, var_return = 'numpy'),
                     tape.jacobian(target = target, var = k, var_return = 'numpy'))
                else :
                    pred = (tape.jacobian(var = x, var_return = 'numpy'),
                     tape.jacobian(var = v, var_return = 'numpy'),
                     tape.jacobian(var = k, var_return = 'numpy'))
                answer = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answer, pred, tor)
                tape.resetgrads()
        except Exception as e:
            print(f'matrix_order_test_{order} is failed :', e) 

    def gradient_start_index_test(self, tor = 0.01, x = None, v = None, k = None):
        try :
            if x is None:
                x = Variable(np.array([[1.,2.],[4.,5.]]))
            if v is None:
                v = Variable(np.array([[4.,5.],[6.,7.]]))
            if k is None:
                k = Variable(np.array([[1.,3.],[4.,6.]]))
            start_index_function = self.start_preset
            function_list = [i for i in start_index_function.__dir__() if not i.startswith('_')]
            for function in function_list:
                current_function = start_index_function.__getattribute__(function)
                pred = current_function(x, v, k)
                answer = self.answer_preset.__getattribute__(function)()
                self.answer_correction(function, answer, pred, tor)
        except Exception as e:
            print(f'{function} is failed :', e) 
    def modeling_test(self, tor = 1e-7, end_iter = 4):
        try:
            X,y = self.data_preprocessing()

            lr = 0.1
            loss_flow = []
            loss_iter = 0

            start_time = time.time()
            model = Models(100, 1)
            optimizers = optimizer.Adam()
            optimizers.setup(model)
            for i in range(10000):
                with GradientTape() as tape:
                    y_pred = model(X)

                    loss = f.loss.mean_squared_error(y, y_pred)

                tape.CalcGradient()

                optimizers.update()

                if i % 1000 == 0:
                    loss_flow.append(loss.data)
                    if (loss_iter > end_iter):
                        if (abs(loss_flow[loss_iter - 1]) - (
                            abs(loss_flow[loss_iter]))) < tor:
                            print('loss_value :' , [float(i) for i in loss_flow])
                            print('model is under local minima, Attempt to retry..')
                            self.modeling_test()
                            break
                        else :
                            print('loss_value :' , [float(i) for i in loss_flow])
                            print('model training test : ok')
                            break
                    loss_iter += 1
        except Exception as e:
            print(f'model_training test is failed : {e}')
    def start_testing(self):
        self.high_order_test()
        self.matrix_test()
        self.jacobian_test()
        self.gradient_start_index_test()
        self.modeling_test()
        

In [193]:
test = UnitTest()

In [194]:
test.start_testing()

high_order_test_0  : ok
high_order_test_1  : ok
matrix_order_test_0  : ok
matrix_order_test_1  : ok
matrix_order_test_2  : ok
matmul  : ok
reduce_sum  : ok
div  : ok
sum  : ok
mul  : ok
gradient_start_middle  : ok
gradient_stop_test  : ok
loss_value : [0.13565552984441984, 1.816673328341998e-05, 1.4886899072852413e-05, 4.626298628297973e-06, 3.5902030429081965e-06, 3.0136820419654117e-06]
model training test : ok


In [169]:
test.high_order_test()

high_order_test_0  : ok
high_order_test_1  : ok


In [170]:
test.matrix_test()

matrix_order_test_0  : ok
matrix_order_test_1  : ok
matrix_order_test_2  : ok


In [171]:
test.jacobian_test()

matmul  : ok
reduce_sum  : ok
div  : ok
sum  : ok
mul  : ok


In [172]:
test.gradient_start_index_test()

gradient_start_middle  : ok
gradient_stop_test  : ok


In [173]:
test.modeling_test()

loss_value : [array(0.12923438), array(4.02457493e-06), array(3.32494054e-06), array(3.18126507e-06), array(2.89203548e-06), array(2.65531922e-06)]
model training test : ok


In [67]:
class Models(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l2 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l3 = layer.Linear(out_size, initializer_func='he_uniform')
        
    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(y)
        y = f.activation.relu(y)
        y = self.l3(y)
        return y

In [68]:
model = Models(100, 1)

optimizers = optimizer.Adam()

optimizers.setup(model)

In [69]:
start_time = time.time()
for i in range(10000):
    with GradientTape() as tape:
        y_pred = model(X)
        
        loss = f.loss.mean_squared_error(y, y_pred)
        
    tape.CalcGradient()
    
    optimizers.update()
    
    if i % 1000 == 0:
        print(loss.data)

print('total_time : ', time.time() - start_time)

0.13316079589785426
4.678867569526384e-06
3.323818075624142e-06
2.434985858024435e-06
1.7888589763894105e-06
1.3856699026652211e-06
1.1175656879502677e-06
9.268755179942446e-07
8.51791997634991e-07
6.456100152307053e-07
total_time :  12.611826419830322


In [318]:
a = np.array([1,2])

In [24]:
b = cp.asarray(a)

In [39]:
a_cp = cp.get_array_module(a.data)

In [42]:
a_cp.array(a.data)

array([[1, 3],
       [4, 5],
       [6, 8]])

In [20]:
cp.arange(5).reshape(2,3)

ValueError: cannot reshape array of size 5 into shape (2, 3)